In [1]:
## ! pip install PyMySQL
## ! pip install ipython-sql
## ! pip install mysqlclient

In [2]:
import requests
from bs4 import BeautifulSoup
from lxml import etree
import pandas as pd
import pymysql

In [3]:
## Creamos una variable con el url a trabajar
r = requests.get('https://volvorepuestos.com.pe/repuestos-camiones-volvo/fm-fmx/suspension.html')

In [4]:
## Validamos que tengamos respuesta correcta con la url
r.status_code

200

In [5]:
# Creamos un variable soup para realizar las busquedas
soup = BeautifulSoup(r.content,'html.parser')

In [6]:
# Validamos que el contenido este correcto
soup

<!DOCTYPE html>

<html lang="es">
<head>
<meta charset="utf-8"/>
<meta content="INDEX,FOLLOW" name="robots"/>
<meta content="Suspensión - FM - FMX - Camiones Volvo" name="title"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1.0, user-scalable=no" name="viewport"/>
<meta content="telephone=no" name="format-detection"/>
<title>Suspensión - FM - FMX - Camiones Volvo</title>
<link href="https://volvorepuestos.com.pe/static/version1574543389/frontend/Smartwave/porto/es_PE/mage/calendar.min.css" media="all" rel="stylesheet" type="text/css"/>
<link href="https://volvorepuestos.com.pe/static/version1574543389/frontend/Smartwave/porto/es_PE/css/styles-m.min.css" media="all" rel="stylesheet" type="text/css"/>
<link href="https://volvorepuestos.com.pe/static/version1574543389/frontend/Smartwave/porto/es_PE/owl.carousel/assets/owl.carousel.min.css" media="all" rel="stylesheet" type="text/css"/>
<link href="https://volvorepuestos.com.pe/static/version1574543389/frontend/Smartw